In [434]:
import os
import re
import pandas as pd
import datetime
import numpy as np
import math
import time


In [435]:
#делаем выгрузки для 4х баз ниже стандартно, удаляем анализы и вакцины

au = pd.read_excel('Выгрузки/АУ.xlsx', dtype={'Дата приема': str, 'UF_USER' : str})

bdvu = pd.read_excel('Выгрузки/БДВУ.xlsx', dtype={'UF_USER' : str, 'Номер карты' : str, 'Дата приема': str})

vu = pd.read_excel('Выгрузки/ВУ.xlsx', dtype={'UF_CRM_1571145004' : str, 'Номер карты' : str, 'Дата выезда': str})

timetable = pd.read_excel('Выгрузки/Расписание.xlsx', dtype={'UF_USER' : str})



In [436]:
au['Дата приема'], bdvu['Дата приема'], vu['Дата выезда'] = pd.to_datetime(au['Дата приема']).dt.date, pd.to_datetime(bdvu['Дата приема']).dt.date, pd.to_datetime(vu['Дата выезда'], format='%d.%m.%Y').dt.date

au['АУ/ВУ'] ='АУ'
bdvu['АУ/ВУ'] ='ВУ'
vu['АУ/ВУ'] ='ВУ'

In [437]:
col_names = [i for i in au.columns if i != 'Дата первого приема Исполнителя']

dct_bdvu, dct_vu = dict(zip(bdvu.columns, col_names)), dict(zip(vu.columns, col_names))

bdvu.rename(columns=dct_bdvu, inplace=True)
vu.rename(columns=dct_vu, inplace=True)




In [438]:
au_vu = pd.concat([au, bdvu, vu]).reset_index(drop=True)

In [439]:
#забираем новых врачей из файла от отдела персонала

personal = pd.read_excel(r'C:\Users\AmanovRA\Текущее2\Новые врачи\новые врачи с 01 03 2020.xls', skiprows=1, dtype={'Дата приема' : str})

personal['Сотрудник'][np.where(personal['Сотрудник'] == 'Краснощёкова Елена Васильевна')[0][0]] = \
'Краснощекова Е.В.' # поменял ё на е чтобы в базах совпало


C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [440]:
number_doctors = au_vu.drop_duplicates(subset=['UF_USER'], ignore_index=True)

personal = personal.merge(number_doctors[['Доктор', 'UF_USER']], left_on='Сотрудник', right_on='Доктор', how='left')

personal.rename(columns={'Дата приема' : 'Дата приема на работу'}, inplace=True)

personal['Дата приема на работу'] = pd.to_datetime(personal['Дата приема на работу']).dt.date

In [441]:
personal['Отработано полных месяцев'] = ((datetime.datetime.now().date() - personal['Дата приема на работу']) / np.timedelta64(1, 'M')).astype(int)

In [442]:
personal.index = personal['UF_USER']

In [443]:
au_vu = au_vu[au_vu['UF_USER'].isin(personal['UF_USER'])].reset_index(drop=True)

In [444]:
au_vu_indicators = au_vu.groupby('UF_USER').agg({'Дата приема' : 'nunique', 'Sum([Количество])' : sum, 'Номер карты' : 'nunique'}).rename(columns = {'Дата приема' : 'Кол-во смен', 'Sum([Количество])' : 'Общее кол-во оказанных услуг', 'Номер карты' : 'Кол-во пациентов'})

In [445]:
category = pd.pivot_table(au_vu, index=['UF_USER'], values='Sum([Количество])', columns='Категория крупно', aggfunc=sum, fill_value=0)


In [446]:
category['Все услуги'] = sum([category[i] for i in category.columns])
category['Первичные консультации'] = sum([category[i] for i in category.columns if 'консуль' in i.lower() and 'повтор' not in i.lower()])
category['Повторные консультации'] = sum([category[i] for i in category.columns if 'консуль' in i.lower() and 'повтор' in i.lower()])
category['Консультации'] = category['Первичные консультации'] + category['Повторные консультации']
category['% первичных консультаций'] = (category['Первичные консультации'] / category['Консультации'])
category['% повторных консультаций'] = (category['Повторные консультации'] / category['Консультации'])
category['% манипуляций ко всего услуг'] = (category['врачебные манипуляции'] / category['Все услуги'])
category = category.rename(columns = {'вакцинация' : 'Кол-во вакцинаций (для педиатров)'})
category = category[['Кол-во вакцинаций (для педиатров)', 'врачебные манипуляции', 'Первичные консультации', 'Повторные консультации', 'Консультации', '% первичных консультаций', '% повторных консультаций','% манипуляций ко всего услуг']]


In [447]:
visits = au_vu.groupby(['UF_USER', 'Номер карты'], as_index=False).agg({'Дата приема' : 'nunique'}).rename(columns={'Дата приема' : 'Кол-во визитов'})

In [448]:
visits['Категория визита'] = pd.cut(visits['Кол-во визитов'], [0, 1, 2, 3, np.inf], labels=['1 посещение', '2 посещения', '3 посещения', '4 и более посещений'])

In [449]:
visits_count = pd.pivot_table(visits, index=['UF_USER'], values='Номер карты', columns='Категория визита', aggfunc='count', fill_value=0)

In [450]:
visits_count.columns = visits_count.columns.add_categories('Все посещения')
visits_count['Все посещения'] = visits_count.sum(axis=1)


In [451]:
for column in [i for i in visits_count.columns if i != 'Все посещения']:
    visits_count[column] = visits_count[column] / visits_count['Все посещения']


In [452]:
visits_count = visits_count.rename(columns = {'1 посещение' : '% пациентов с 1 посещением', '2 посещения' : '% пациентов с 2 посещениями', '3 посещения' : '% пациентов с 3 посещениями', '4 и более посещений' : '% пациентов с 4 и более посещениями'})

In [453]:
visits_count = visits_count[[i for i in visits_count.columns if i != 'Все посещения']]


In [454]:
personal = personal.join([au_vu_indicators, category, visits_count])

In [455]:
personal['Услуг за смену'] = personal['Общее кол-во оказанных услуг'] / personal['Кол-во смен']

In [456]:
timetable = timetable.drop_duplicates(subset=['UF_USER'], ignore_index=True)

timetable = timetable[timetable['UF_USER'].isin(personal.index)].reset_index(drop=True)

In [457]:
timetable['Загрузка'] = timetable['Записи с лечением + доп талоны'] / timetable['Доступно для записи Пациентов']
timetable.index = timetable['UF_USER']
timetable.drop('UF_USER', axis=1, inplace=True)

In [458]:
personal = personal.merge(timetable[['Доступно для записи Пациентов', 'Записи с лечением + доп талоны', 'Загрузка']], left_index=True, right_index=True, how='left')

personal = personal.fillna(0)

# Результаты обзвона

In [459]:
temp_names = personal[['Сотрудник']]
temp_names['Фамилия'] = personal['Сотрудник'].str.split().str.get(0)


C:\Users\AmanovRA\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [460]:
temp_dct = {}
for file in os.listdir('./Обзвоны'):
    if '$' not in file:
        name = file.split()[1].strip()
        temp_dct.setdefault(name, {})
        temp_df = pd.read_excel('./Обзвоны/' + file, header=0)
        temp_df = temp_df[[col for col in temp_df.columns if any(['бал' in str(col).lower(), 'ценк' in str(col).lower()])]]
        temp_df = temp_df.dropna()
        col_name = temp_df.columns[0]
        arr = np.array([i for i in temp_df[col_name].values if isinstance(i, (int, float, np.int64))])
        temp_dct[name]['Больше или равно 7 баллов'] = arr[arr >= 7].shape[0]
        temp_dct[name]['Меньше 7 баллов'] = arr[arr < 7].shape[0]
        temp_dct[name]['Обзвонено клиентов'] = arr.shape[0]
        
temp_df = pd.DataFrame(temp_dct)
temp_df = temp_df.T


In [461]:
temp_names = temp_names.merge(temp_df, left_on='Фамилия', right_index=True, how='left').fillna('Нет данных')

In [462]:
personal = personal.merge(temp_names[['Больше или равно 7 баллов', 'Меньше 7 баллов', 'Обзвонено клиентов']], left_index=True, right_index=True)

In [466]:
personal = personal[['Подразделение 2-го уровня', 'Подразделение 3-го уровня', 'Подразделение 4-го уровня', 'Должность УУ', 'Сотрудник', 'Вид занятости', 'Дата приема на работу', 'Отработано полных месяцев',  'Кол-во смен', 'Услуг за смену', 'Общее кол-во оказанных услуг',  'врачебные манипуляции', 'Первичные консультации', 'Повторные консультации', '% манипуляций ко всего услуг', '% первичных консультаций', '% повторных консультаций', 'Кол-во пациентов',  '% пациентов с 1 посещением', '% пациентов с 2 посещениями', '% пациентов с 3 посещениями', '% пациентов с 4 и более посещениями', 'Доступно для записи Пациентов', 'Записи с лечением + доп талоны', 'Загрузка', 'Обзвонено клиентов', 'Больше или равно 7 баллов', 'Меньше 7 баллов']]

In [468]:
personal.to_excel('Отчет_новые_врачи_октябрь.xlsx', index=False)